In [67]:
import chatterbot
import os
import json
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
from spacy import displacy
from collections import Counter
#!python -m spacy download en_core_web_trf

In [68]:
def fix_text(text):
    text = text.replace("â\x80\x99", "'")
    if "ð" in text:
        for word in text.split():
            if "ð" in word:
                text = text.replace(word, "EMOJI")
    return text


#This chunk of code reads the json files for FACEBOOK MESSENGER MESSAGES
person = "Amit" #Which person do you want the chat bot to replicate?
sender = "Amit Krishnaiyer"
path = "../Messages/" + person
conversation_list = [] #List of all conversations in an organized format
speaker_list = [] #List of everything the speaker said
for filename in os.listdir(path):
    if filename[-4:] == "json":
        conversation = []
        f = open(path + "/" + filename)
        data = json.load(f)["messages"]
        for message in data:
            if "content" in message:
                the_message = fix_text(message["content"])
                conversation.append((the_message, message["sender_name"]))
                if sender == message["sender_name"]:
                    speaker_list.append(the_message)
        conversation_list.append(conversation)


In [69]:
#This block of code gets the speaker's most common used words
word_lis = []
for message in speaker_list:
    message = preprocesser(message, True, False, False, True, True)# (text, lower, punctuation, named_entity, white_space, stop_words)
    words = message.split()
    for word in words:
        if word != "":
            word_lis.append(word)

# Pass the split_it list to instance of Counter class.

Counter = Counter(word_lis)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(30)
  
print(most_occur)
        
    
    

[('amit', 132), ('played,', 115), ('u', 111), ('t', 82), ('yea', 80), ('ur', 74), ('cuz', 66), ('like', 65), ('lol', 53), ('get', 52), ("i'm", 49), ('n', 47), ('also', 42), ('lke', 42), ('thnk', 38), ('rlly', 38), ('w', 37), ('ab', 29), ('emoji', 28), ('go', 28), ('well', 27), ('tho', 27), ("'m", 26), ('good', 26), ('f', 26), ('would', 25), ('one', 25), ('got', 24), ('see', 24), ('even', 24)]


In [66]:
'''
Only have to run the code below once
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
'''

lemmatizer = WordNetLemmatizer()

stop_words_set = stopwords.words('english')
custom_stop_words = ["um", "uhm", "uh", "oh", "ok", "turn!"]
stop_words_set = set(stop_words_set + custom_stop_words)



def find_and_replace(sentence):
    dic = {}
    for sent in nltk.sent_tokenize(sentence):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
             if hasattr(chunk, 'label'):
                #print(f"{' '.join(c[0] for c in chunk):<35} {chunk.label()}")
                thing = ' '.join(c[0] for c in chunk)
                dic[thing] = chunk.label()
                #print(thing, chunk.label())
    for key in dic:
        sentence = sentence.replace(key, dic[key])
    return sentence

def remove_punctuation(word):
    if word[0].isalpha() == False and word[0].isdigit() == False:
        word = word[1:]
    if len(word) > 0:
        if word[-1].isalpha() == False and word[-1].isdigit() == False:
            word = word[:-1]
    return word


def preprocesser(text, lower, punctuation, named_entity, white_space, stop_words):
    if lower:
        text = text.lower()
    if punctuation:
        text = remove_punctuation(text)
    if named_entity:
        text = find_and_replace(text)
    if white_space:
        text = text.strip()
    if stop_words:
        for word in text.split():
            if word in stop_words_set:
                text = text.replace(word, "")    
    return text

sentence = "The Kashmir Files is a 2022 Indian Hindi-language drama film, \
written and directed by Vivek Agnihotri. Produced by Zee Studios, \
the film is based on the exodus of Kashmiri Pandits during the Kashmir Insurgency, \
which it portrays as a genocide. \
It stars Anupam Kher, Darshan Kumar, Pallavi Joshi and Mithun Chakraborty."
print(find_and_replace(sentence))     
    

The ORGANIZATION is a 2022 GPE Hindi-language drama film, written and directed by PERSON. Produced by PERSON, the film is based on the exodus of ORGANIZATION during the ORGANIZATION, which it portrays as a genocide. It stars PERSON, PERSON, PERSON and PERSON.


In [55]:
from chatterbot import ChatBot
chatbot = ChatBot(person)

preprocessors=['chatterbot.preprocessors.clean_whitespace']

[nltk_data] Downloading package stopwords to /Users/amit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/amit/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [56]:
from chatterbot.trainers import ListTrainer

trainer = ListTrainer(chatbot)

trainer.train([
    "Hi there!",
    "Hello!!",
])

trainer.train([
    "Greetings!",
    "Hello!",
])
response = chatbot.get_response("Greetings!")
print(response)


'''
What do I consider when responding to texts?
High Priority:
Energy of the message: I try to match energy: Could measure energy of message?
What is the sentiment of the message?
What are my interests at the moment?
Who are the people I'm talking about the most at the time. 


Bonus: What are they asking for- is it something I can google?


Low Priority:
What time of day is it? Do I text differently at night than day? 

'''

List Trainer: [####################] 100%
List Trainer: [####################] 100%
Hello!


"\nWhat do I consider when responding to texts?\nHigh Priority:\nEnergy of the message: I try to match energy: Could measure energy of message?\nWhat is the sentiment of the message?\nWhat are my interests at the moment?\nWho are the people I'm talking about the most at the time. \n\n\nBonus: What are they asking for- is it something I can google?\n\n\nLow Priority:\nWhat time of day is it? Do I text differently at night than day? \n\n"